In [2]:
import itertools
import logging
import os
from functools import partial

import openai
import pandas as pd
from dotenv import load_dotenv
import lastmile_utils.lib.jupyter as jupyter_utils

pd.set_option("display.max_colwidth", None)

jupyter_utils.set_log_level(logging.WARNING)


load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


from aiconfig.eval.lib import (
    TestSuiteWithInputsConfig,
    UserTestSuiteWithInputs,
    brevity,
    contains_substring,
    run_test_suite_with_inputs,
)

tsconfig = TestSuiteWithInputsConfig(
    {
        "prompt_name": "get_activities",
        "aiconfig_path": "./travel_parametrized.aiconfig.json",
    }
)


substring_case_insensitive = partial(contains_substring, case_sensitive=False)


test_inputs = [
    "Empire State Building is on fifth avenue. What is the cross street?",
    "What is the best borough?",
]
test_suite_with_inputs = UserTestSuiteWithInputs(
    [
        (test_inputs[0], partial(substring_case_insensitive, substring="34th street")),
        (test_inputs[1], partial(substring_case_insensitive, substring="Brooklyn")),
    ]
    + list(itertools.product(test_inputs, [brevity]))
)


df_result = await run_test_suite_with_inputs(
    test_suite=test_suite_with_inputs,
    config=tsconfig,
)

AssertionError: 

In [2]:
df_result

,input,aiconfig_output,value,metric_name,metric_description
0,Empire State Building is on fifth avenue. What is the cross street?,"The Empire State Building is located on the intersection of Fifth Avenue and West 34th Street in Manhattan, New York City.",1.0,contains_substring,1.0 (pass) if contains given substring
1,Empire State Building is on fifth avenue. What is the cross street?,"The Empire State Building is located on the intersection of Fifth Avenue and West 34th Street in Manhattan, New York City.",122.0,brevity,Absolute text length
2,What is the best borough?,"As an AI, I don't have personal opinions. However, the ""best"" borough can vary depending on individual preferences and needs. Some popular options include Manhattan for its vibrant city feel and iconic landmarks, Brooklyn for its diverse neighborhoods and artistic communities, and Queens for its cultural diversity and affordable housing options. Ultimately, the ""best"" borough is subjective and depends on factors such as lifestyle, amenities, and personal preferences.",1.0,contains_substring,1.0 (pass) if contains given substring
3,What is the best borough?,"As an AI, I don't have personal opinions. However, the ""best"" borough can vary depending on individual preferences and needs. Some popular options include Manhattan for its vibrant city feel and iconic landmarks, Brooklyn for its diverse neighborhoods and artistic communities, and Queens for its cultural diversity and affordable housing options. Ultimately, the ""best"" borough is subjective and depends on factors such as lifestyle, amenities, and personal preferences.",471.0,brevity,Absolute text length


In [3]:
df_result.set_index(["input", "aiconfig_output", "metric_name"])\
        .value.unstack("metric_name")

,metric_name,brevity,contains_substring
input,aiconfig_output,,
Empire State Building is on fifth avenue. What is the cross street?,"The Empire State Building is located on the intersection of Fifth Avenue and West 34th Street in Manhattan, New York City.",122.0,1.0
What is the best borough?,"As an AI, I don't have personal opinions. However, the ""best"" borough can vary depending on individual preferences and needs. Some popular options include Manhattan for its vibrant city feel and iconic landmarks, Brooklyn for its diverse neighborhoods and artistic communities, and Queens for its cultural diversity and affordable housing options. Ultimately, the ""best"" borough is subjective and depends on factors such as lifestyle, amenities, and personal preferences.",471.0,1.0
